
# 🎯 Actividad Práctica — Sesión 5 (45–50 min)

**Duración total sugerida:** 50–60 min


> 📝 **Nota :**  
> El código que se muestra a continuación está intencionadamente presentado en **celdas de texto (Markdown)**.  
> La idea es que usted pueda **copiarlo, adaptarlo y crear su propio notebook**, tomando decisiones sobre la estructura de la CNN y los hiperparámetros.  
>
> Este enfoque busca promover la comprensión profunda del modelo y no solo su ejecución.

Este ejercicio busca que usted analice la arquitectura de una CNN simple y explore cómo los hiperparámetros afectan su comportamiento.


## PARTE 1: Entendiendo una CNN simple y sus hiperparámetros

En esta primera parte construiremos una CNN pequeña utilizando el dataset MNIST.
El objetivo es comprender la estructura básica de una red convolucional y reflexionar sobre el impacto de modificar sus hiperparámetros.

```python
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

# Cargar datos MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalizar y adaptar formato (28x28x1)
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

# Añadir dimensión de canal (blanco y negro)
x_train = x_train[..., tf.newaxis]   # (60000, 28, 28, 1)
x_test  = x_test[..., tf.newaxis]

num_clases = 10




### Ahora que tenemos los datos listos, construiremos una CNN simple para identificar sus partes fundamentales y sus respectivos hiperparámetros modificables.
```python
# MODELO CNN SIMPLE
modelo_cnn = models.Sequential([
    # Capa convolucional 1
    layers.Conv2D(
        filters=32,          # ← hiperparámetro 1
        kernel_size=(3, 3),  # ← hiperparámetro 2
        activation="relu",
        input_shape=(28, 28, 1)
    ),
    layers.MaxPooling2D(pool_size=(2, 2)),  # ← hiperparámetro 3

    # Capa convolucional 2
    layers.Conv2D(
        filters=64,          # ← hiperparámetro 4
        kernel_size=(3, 3),
        activation="relu"
    ),
    layers.MaxPooling2D(pool_size=(2, 2)),

    layers.Flatten(),

    layers.Dense(64, activation="relu"),     # ← hiperparámetro 5 (número de neuronas)
    layers.Dense(num_clases, activation="softmax")
])

# Compilación del modelo
modelo_cnn.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Resumen de la arquitectura
modelo_cnn.summary()


### La siguiente sección permite configurar la cantidad de iteraciones (épocas) que ejecutará el modelo.

```python
historial = modelo_cnn.fit(
    x_train, y_train,
    epochs=2,              # ← hiperparámetro 6
    batch_size=128,        # ← hiperparámetro 7
    validation_split=0.1,
    verbose=1
)


###Considerando solo la vista -y no ejecucion del codigo - reflexione a partir de la sig uiente pregunta:  ¿Qué cree que pasaría si…?

* Duplicamos filters en la primera capa de 32 → 64.

* Quitamos una capa de MaxPooling2D.

* Aumentamos kernel_size de (3,3) a (5,5).

* Aumentamos la capa densa final de 64 → 256 neuronas.

* Aumentamos epochs de 2 → 10.

### ¿Cómo cree que cambiaría…?

* El tiempo de entrenamiento.

* El riesgo de sobreajuste (overfitting).

* La capacidad del modelo para aprender patrones más complejos.

### Modifique 1 o 2 hyperparametros y ejecute el entrenamiento muy breve (1–2 epochs).

* Compare el accuracy y el tiempo de entrenamiento.

### Glosario de Hiperparámetros

A continuación se describen los principales hiperparámetros utilizados en la CNN simple construida en esta actividad:

**1. `filters`**  
Número de filtros (o kernels) en una capa convolucional.  
Más filtros permiten detectar más tipos de patrones, pero aumentan el costo computacional y el riesgo de sobreajuste.

**2. `kernel_size`**  
Tamaño del filtro que “recorre” la imagen (ej.: 3×3, 5×5).  
Un kernel más grande capta patrones más amplios, pero puede perder detalle y hacer el modelo más lento.

**3. `pool_size`**  
Tamaño de la ventana usada en MaxPooling.  
Reduce la dimensión espacial del mapa de características, haciendo el modelo más eficiente y menos propenso al sobreajuste.

**4. `filters` (segunda capa)**  
Igual que el hiperparámetro 1, pero en una capa más profunda.  
Generalmente se aumenta (32→64→128) para aprender patrones más complejos.

**5. `units` en `Dense`**  
Número de neuronas en la capa completamente conectada.  
Más neuronas permiten representar relaciones más complejas, pero incrementan el riesgo de sobreajuste.

**6. `epochs`**  
Cantidad de pasadas completas por todo el conjunto de entrenamiento.  
Más épocas permiten aprender más, pero pueden llevar a sobreentrenamiento.

**7. `batch_size`**  
Cantidad de muestras procesadas por el modelo en cada actualización de parámetros.  
Batches pequeños → aprendizaje más ruidoso pero potencialmente mejor generalización.  
Batches grandes → entrenamiento más estable y rápido, pero con riesgo de sobreajuste.


## PARTE 2: Explorando EfficientNetB0 y sus hiperparámetros

En esta segunda parte trabajaremos con **EfficientNetB0**, un modelo moderno de *Transfer Learning* entrenado originalmente sobre el dataset ImageNet.

El objetivo es identificar sus componentes principales, reconocer los hiperparámetros más relevantes y reflexionar sobre cómo estos afectan el comportamiento del modelo.

> 📝 Nota para participantes  
> Al igual que en la sección anterior, el siguiente código se presenta en formato de texto (Markdown).  
> Se espera que usted **cree su propio notebook** y copie, adapte o modifique este modelo según los cambios que desee explorar.

### Paso 1: Cargar EfficientNetB0 preentrenado

```python
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models, optimizers

# Cargar EfficientNetB0 sin la parte final (include_top=False)
base_model = EfficientNetB0(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)    # ← hiperparámetro 1 (tamaño de entrada)
)

# Congelar la base convolucional (no se entrena)
base_model.trainable = False      # ← hiperparámetro 2 (capas entrenables)


### Paso 2: Crear un nuevo clasificador final
```python
inputs = layers.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)        # ← hiperparámetro 3 (tasa de dropout)
outputs = layers.Dense(5, activation="softmax")(x)   # ← hiperparámetro 4 (número de clases)

modelo_eff = models.Model(inputs, outputs)


### Paso 3: Compilar el modelo
```python
modelo_eff.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),  # ← hiperparámetro 5 (learning rate)
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

modelo_eff.summary()


### Sin ejecutar el código, reflexione a partir de la siguiente pregunta: ¿Qué cree que ocurriría si…?

* Cambiamos input_shape de (224,224,3) a (128,128,3).

* Descongelamos parcialmente la base (base_model.trainable = True).

* Aumentamos el Dropout de 0.2 a 0.5.

* Cambiamos la tasa de aprendizaje de 1e-3 a 1e-5.

* Aumentamos la capa final de 5 a 10 clases.

### ¿Cómo cree que cambiarían…?

* El tiempo de entrenamiento.

* El riesgo de sobreajuste (overfitting).

* La capacidad del modelo para generalizar.

* La estabilidad del gradiente al entrenar.

### Glosario de Hiperparámetros en EfficientNetB0

A continuación se describen los hiperparámetros principales utilizados en el modelo EfficientNetB0 presentado en esta actividad:

**1. `input_shape`**  
Dimensión de entrada esperada por el modelo (ej.: 224×224×3).  
Controla cuánto debe redimensionarse cada imagen antes de entrar a la red.  
Tamaños menores reducen el costo computacional pero pueden perder detalle importante.

**2. `base_model.trainable`**  
Indica si las capas convolucionales preentrenadas pueden actualizarse durante el entrenamiento.  
- `False` → solo se entrena la "cabeza" final (más rápido y estable).  
- `True` → se ajustan todas las capas (mayor capacidad, pero también mayor riesgo de sobreajuste y necesidad de datos).

**3. `Dropout`**  
Fracción de neuronas que se “apagan” aleatoriamente durante el entrenamiento.  
Ayuda a prevenir sobreajuste, especialmente en modelos grandes.

**4. `Dense(units)` (número de clases)**  
Cantidad de neuronas en la capa final.  
Debe coincidir con el número de clases a predecir.  
Un número mayor no implica necesariamente mejor rendimiento.

**5. `learning_rate`**  
Velocidad con la que se actualizan los pesos del modelo durante el entrenamiento.  
- Valores altos → aprendizaje rápido pero inestable.  
- Valores muy bajos → aprendizaje lento y posible estancamiento.

**6. `optimizer`** (Adam en este caso)  
Algoritmo que ajusta los pesos para minimizar la función de pérdida.  
Adam combina ventajas de AdaGrad y RMSProp y funciona bien en la mayoría de problemas.

**7. `GlobalAveragePooling2D`**  
Reduce cada mapa de características a un único valor promedio.  
Disminuye drásticamente el número de parámetros y ayuda a evitar sobreentrenamiento.

**8. `include_top=False`**  
Indica que no se utiliza la parte final del modelo preentrenado (clasificador original de ImageNet).  
Permite agregar un clasificador propio para un nuevo conjunto de clases.



## ¿EfficientNetB0 es una CNN?

Sí. EfficientNetB0 es una red neuronal convolucional (CNN), igual que la que construimos en la Parte 1.  
La diferencia es que EfficientNetB0 es **mucho más profunda y compleja**, e incluye:
- decenas de capas convolucionales,
- capas avanzadas como MBConv y SE blocks,
- normalización,
- etapas de reducción progresiva del tamaño espacial.

En esta actividad no analizamos cada capa internamente (sería demasiado complejo), sino que
utilizamos EfficientNetB0 como **modelo preentrenado** para aplicar Transfer Learning.


## Nota importante: ¿Qué hace realmente EfficientNetB0?

En la actividad anterior, usted utilizó **MobileNetV2** para clasificar imágenes reales de su computador.  
Gracias a eso, pudo observar directamente:

- las predicciones del modelo,  
- los aciertos y errores,  
- los patrones que reconoce,  
- y los sesgos que pueden aparecer.

En esta segunda parte, el objetivo es distinto:  
**no buscamos volver a clasificar imágenes**, sino comprender la estructura interna de un modelo moderno, **EfficientNetB0**, y explorar algunos de sus hiperparámetros clave.

**EfficientNetB0** es también una **red neuronal convolucional (CNN)**, igual que **MobileNetV2**, pero mucho más profunda y compleja.  
Aquí nos enfocamos en:

- cómo está construida su base convolucional,
- qué significa congelar o descongelar capas,
- cómo influyen hiperparámetros como `input_shape`, `dropout` o `learning_rate`,
- y cómo estos cambios afectan el proceso de entrenamiento.

Si aun así desea **comprobar que EfficientNetB0 puede clasificar imágenes reales**, tal como lo hizo MobileNetV2, puede hacerlo ejecutando la celda de código que se encuentra más abajo.  
Esa celda permite cargar una imagen desde su computador y obtener una predicción con el modelo que acaba de construir.


```python
# ================================================
# CLASIFICAR IMÁGENES REALES CON EfficientNetB0
# ================================================

import numpy as np
from tensorflow.keras.preprocessing import image
from google.colab import files
import matplotlib.pyplot as plt

# 1. Subir imagen desde el computador
uploaded = files.upload()

for fn in uploaded.keys():
    img_path = fn
    print("Imagen cargada:", fn)

    # 2. Cargar y preprocesar la imagen
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    # 3. Mostrar imagen cargada
    plt.imshow(image.load_img(img_path))
    plt.axis("off")
    plt.show()

    # 4. Obtener predicción (probabilidades)
    pred = modelo_eff.predict(x)
    print("\nVector de probabilidades:", pred)

    # 5. Mostrar la clase predicha
    pred_class = np.argmax(pred)
    print(f"\nClase predicha: {pred_class}")


Si opto por clasificar imagenes, lo mas probable es que las siguientes preguntas lo respresenten en un 100%:
* ¿Por qué esta CNN tiene solo 5 clases?
* ¿Cómo se interpreta el vector de probabilidades?

En esta actividad, la capa final de **EfficientNetB0** fue definida así:

```python
outputs = layers.Dense(5, activation="softmax")(x)
```
Esto significa que hemos creado un clasificador con 5 categorías posibles.
El modelo no conoce los nombres reales de esas clases (por ejemplo “perro”, “auto”, etc.).
Solo sabe que debe asignar una probabilidad a cada una de las 5 clases numeradas:
* Clase 0
* Clase 1
* Clase 2
* Clase 3
* Clase 4

Ahora bien, cuando ejecutamos:
```python
pred = modelo_eff.predict(x)
```
obtendremos un vector como:
```python
[0.02, 0.65, 0.10, 0.20, 0.03]
```
Cada número representa la probabilidad estimada de que la imagen pertenezca a una de las clases:
* 0.02 : probabilidad de ser clase 0
* 0.65 : probabilidad de ser clase 1
* 0.10 : probabilidad de ser clase 2
* 0.20 : probabilidad de ser clase 3
* 0.03 : probabilidad de ser clase 4

La clase predicha es aquella con la probabilidad más alta:
```python
np.argmax(pred)   # en este caso, 1
```
Es decir, el modelo clasifica la imagen como clase 1.